In [1]:
import pandas as pd
import matplotlib.ticker as tick
import matplotlib.pylab as plt
import seaborn as sns

from util.reformat_large_tick_values import reformat_large_tick_values
from sqlalchemy.engine import create_engine

engine = create_engine('bigquery://', credentials_path= 'key/iowa-liquor-sales-365322-12ff7a5498e6.json')

# Introduction:  

This independent exploratory analysis uses public data on liquor sold in Iowa and US census population data to examine sales trends. This analysis focuses on the ten most populous Iowa counties and the time period 2018-2021. 

The ten most populous Iowa counties (out of 100 total counties) comprise 52.6% of the adult (18+) population and represent 63% of the state's total liquor sales (by volume).

## Public Datasets

Iowa Liquor Sales dataset, Iowa Department of Commerce, Alcoholic Beverages Division
`bigquery-public-data.iowa_liquor_sales`  
Data represents sales to Iowa Class "E" liquor licensees. Examples include grocery stores, liquor stores etc. which sell liquor for off-premises consumption.


Population data, United States Census Bureau, https://www.census.gov/quickfacts/fact/table/IA/POP010220
Accurate population numbers are limited to 2020, the year of the US census. In this analysis, population counts and percent minors for 2020 are used for all years.

# Cleaning

## Null check

In [2]:
#Null check on Iowa Liquor Sales dataset column: volume_sold_liters
null_check_liters_sold_sql_statement = """
SELECT 
  countif(volume_sold_liters is null) / count(1) * 100 as percent_null_liters_sold
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023)
  """.strip()
percent_null_liters_sold = pd.read_sql_query(null_check_liters_sold_sql_statement, engine)
percent_null_liters_sold

/Users/anooshmoutafian/anaconda3/lib/python3.10/site-packages/google/cloud/bigquery/client.py:581: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


,percent_null_liters_sold
0,0.0


In [7]:
#Null check on Iowa Liquor Sales dataset column: county
null_check_county_sql_statement = """SELECT 
  countif(county is null) / count(1) * 100 as percent_null_county
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
WHERE EXTRACT(YEAR from date) IN (2017, 2018, 2019, 2020, 2021, 2022, 2023)
  """.strip()
percent_null_county = pd.read_sql_query(null_check_county_sql_statement, engine)
percent_null_county

,percent_null_county
0,0.304728


.3% of values in the county column are null. The store_number column has no nulls. There are 2,299 distinct store numbers, only two are missing valid county name. 

Strategy: Clean county column by mapping each store number to a valid county name.

In [9]:
cleaned_null_check_county_sql_statement = """
WITH 
source_table AS (
  SELECT
    volume_sold_liters,
    date,
    EXTRACT(YEAR FROM date) AS year,
    IF (county ='POTTAWATTA','POTTAWATTAMIE', county) AS county,
    store_number
  FROM `bigquery-public-data.iowa_liquor_sales.sales`
  WHERE EXTRACT(YEAR FROM date) IN (2017, 2018, 2019, 2020, 2021,2022, 2023)
  ),

store_county_map AS ( 
  SELECT 
  store_number,
  MAX(county) AS county
  FROM source_table
  WHERE store_number IS NOT NULL
  GROUP BY store_number
  ),

joined_table AS (
  SELECT a.volume_sold_liters, a.date, a.year, a.store_number, COALESCE(a.county, b.county) AS county
  FROM source_table a
  LEFT JOIN store_county_map b ON a.store_number = b.store_number
  ),

 null_check_county AS (
   SELECT countif(county is null) / count(1) * 100 as percent_null_county
FROM joined_table)

SELECT * FROM null_check_county
  """.strip()

cleaned_percent_null_county = pd.read_sql_query(cleaned_null_check_county_sql_statement, engine)
cleaned_percent_null_county

,percent_null_county
0,0.006698


County column nulls reduced from 0.304728% to  0.006698%

Adult population was extrapolated using the "Persons under 18 years, percent" census data column as no counts are available for 21 years and older.